# Answering Business Questions using SQL
In this project we'll be working with a modified version of a database called Chinook. The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. [Here](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)'s a schema diagram for the Chinook database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql

SELECT * FROM customer
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [4]:
%%sql

SELECT * FROM invoice
LIMIT 5;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


## Selecting Albums to Purchase
The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four:

|Artist Name           |	Genre |
|----------------------|----------|
|Regal	               | Hip-Hop  |
|Red Tone              | Punk     |
|Meteor and the Girls  | Pop      |
|Slim Jim Bites	       | Blues    |

The record label specializes in artists from the USA, so we're interested in finding out which genres sell the best in the USA.

In [5]:
%%sql

WITH 
    track_info AS
    (
        SELECT t.*
        FROM track t 
        JOIN invoice_line il ON il.track_id = t.track_id
        JOIN invoice i ON i.invoice_id = il.invoice_id
        WHERE i.billing_country = "USA"
    ),
    
    genre_info AS
    (
    SELECT 
        g.name genre,
        COUNT(ti.track_id) tracks_sold
    FROM track_info ti
    JOIN genre g ON g.genre_id = ti.genre_id
    GROUP BY 1
    ORDER BY 2 DESC
    )
    
SELECT 
    *,
    ROUND(CAST(tracks_sold as Float) / (
        SELECT SUM(tracks_sold) FROM genre_info
        ) * 100, 2) tracks_sold_pct
FROM genre_info;

Done.


genre,tracks_sold,tracks_sold_pct
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


According to the table, the most sold genre is Punk and the least is Hip-Hop. So based on sales of tracks from their genres, we should purchase for the store:
- Red Tone - Punk
- Slim Jim Bites - Blues
- Meteor and the Girls - Pop

## Analyzing Employee Sales Performance

In [6]:
%%sql

WITH
    sales_support_agents AS
    (
    SELECT 
        employee_id,
        first_name || ' ' || last_name employee_name,
        hire_date 
    FROM employee
    WHERE title = 'Sales Support Agent'
    )

SELECT 
    ssa.employee_name,
    ssa.hire_date,
    SUM(i.total) total_sales
FROM sales_support_agents ssa
JOIN customer c ON c.support_rep_id = ssa.employee_id
JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1,2
ORDER BY 3 DESC

Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


There is a difference in total dollar amount of sales assigned to each sales support agent but possible explanation is the difference in the hiring dates: the longer an agent is working for the company, the higher their total sales are.

## Analyzing Sales by Country

In [26]:
%%sql

WITH 
    countries AS
    (
    SELECT
        DISTINCT c.customer_id,
        CASE
            WHEN (
                SELECT COUNT(country)
                FROM customer
                WHERE country = c.country
                ) = 1 THEN 'Other'           
            ELSE c.country
        END AS country,
        il.*
    FROM customer c
    JOIN invoice i ON i.customer_id = c.customer_id
    JOIN invoice_line il ON il.invoice_id = i.invoice_id
    ),
    
    sales_info AS
    (
    SELECT 
        country,
        COUNT(DISTINCT customer_id) num_customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / COUNT(DISTINCT customer_id) avg_sales_per_customer,
        SUM(unit_price) / COUNT(DISTINCT invoice_id) avg_order,
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS sort
    FROM countries
    GROUP BY 1
    ORDER BY sort, total_sales DESC
    )
    
SELECT
    country,
    num_customers,
    total_sales,
    avg_sales_per_customer,
    avg_order
FROM sales_info;

Done.


country,num_customers,total_sales,avg_sales_per_customer,avg_order
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486


Based on analysis of sales by country, the countries with the highest average value of sales per customer are:
- Czech Republic
- Portugal

And the countries with the highest average order value are:
- Czech Republic
- United Kingdom
- India

Although it looks like a good idea to implement marketing campaigns in these countries in order to attract more customers, we shouldn't forget that our sample size is too small to predict the result. It might be better to start with spending small amounts in adveretizing in these countries and then make a disicion based on the response.

## Album vs Individual Tracks

In [41]:
%%sql

WITH last_track AS
    (
    SELECT 
        invoice_id,
        MAX(track_id) last_track_id
    FROM invoice_line
        GROUP BY 1
    )

SELECT 
    album_purchased,
    COUNT(invoice_id) number_of_invoices,
    CAST(COUNT(invoice_id) AS Float) / 
                (
                SELECT COUNT(*) FROM invoice 
                ) * 100 percentage_of_invoices
    
FROM 
    (
    SELECT
        *,
        CASE WHEN 
            (
            SELECT track_id 
            FROM track
            WHERE album_id = (
                                SELECT t2.album_id
                                FROM track t2
                                WHERE t2.track_id = lt.last_track_id
                                )
            EXCEPT
                
            SELECT il2.track_id
            FROM invoice_line il2
            WHERE il2.invoice_id = lt.invoice_id
            ) IS NULL
        AND
            (
            SELECT il2.track_id
            FROM invoice_line il2
            WHERE il2.invoice_id = lt.invoice_id
                
            EXCEPT
                
            SELECT track_id 
            FROM track
            WHERE album_id = (
                                SELECT t2.album_id
                                FROM track t2
                                WHERE t2.track_id = lt.last_track_id
                                )
            ) IS NULL
        
        THEN "True"
        ELSE "False"
        END AS "album_purchased"
        
    FROM last_track lt
    )
    
GROUP BY 1

Done.


album_purchased,number_of_invoices,percentage_of_invoices
False,500,81.43322475570032
True,114,18.566775244299674


Around 18,56% purchase the whole albums. It means the company's strategy of purchasing only popular tracks, instead of every track from an album, may result in losing almost 20% of income.